In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/climate-change-edsa2020-21/sample_submission.csv
/kaggle/input/climate-change-edsa2020-21/test.csv
/kaggle/input/climate-change-edsa2020-21/train.csv


In [2]:
# import libraries

import numpy as np
import pandas as pd
import re
import nltk
from sklearn.datasets import load_files

import string
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer



In [3]:
# read the datasets
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')


In [4]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [5]:
train.head()


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [6]:
# split data into X and y
y = train['sentiment']
X = train['message']

In [7]:
# Pre Processing
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

# Pre processing 
for sen in range(0, len(X)):
    tweet = str(X[sen])
    # 
    # lower case + remove puntuation
    tweet = "".join([word.lower() for word in tweet if word not in string.punctuation])
    # remove all single characters
    tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', tweet)
    
    # Remove single characters from the start
    tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', tweet) 
    
    # Substituting multiple spaces with single space
    tweet = re.sub(r'\s+', ' ', tweet, flags=re.I)
    # renone number
    tweet = re.sub('[0-9]+', '', tweet)
    
    
    #
    #tweet = re.compile(r"https?://(www\.)?")
    
    tweet=re.sub(r'http\S+', '', tweet)
    #tweet=tweet.sub('', 'http://www.google.com/images').strip().strip('/')
    # tokenization
    tweet_tokens = re.split('\W+', tweet)  
    
    # remove stop words
        
    tokens_without_sw = [word for word in tweet_tokens if not word in stopwords.words('english')]
    # lemmatize
    document = [stemmer.lemmatize(word) for word in tokens_without_sw]
    document = ' '.join(document)
    #print(document[:])
    
    #print(tweet)
    
    #tweet = ' '.join(tweet)

    #print(tweet[:])
    
    documents.append(document)
  

In [8]:
# run a few checks
print(documents[0])
print(documents[2])
print(documents[12])

polyscimajor epa chief doesnt think carbon dioxide main cause global warming wait via mashable
rt rawstory researcher say three year act climate change late 
rt tveitdal percent chance avoiding dangerous global warming study find 


In [9]:
# vectorise the data so the model can undertands
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_vectorized = tfidfconverter.fit_transform(documents).toarray()

In [10]:
#split te data into Test and Train

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_vectorized,y,test_size=0.20,shuffle=True, random_state=25)

In [11]:
# To train our machine learning model using the random forest algorithm we will 
# use RandomForestClassifier class from the sklearn.ensemble library. 
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000, random_state=0)
rfc = rfc.fit(X_train, y_train) 

In [12]:
# Generate predicts based on the test split
rfc_pred = rfc.predict(X_test)

In [13]:
# get f1_score
f1_score=f1_score(y_test,rfc_pred, average="macro")

In [14]:
print(f1_score)

0.5679851300525272


In [15]:
from sklearn import metrics

print(metrics.classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

          -1       0.73      0.23      0.36       247
           0       0.57      0.38      0.46       466
           1       0.72      0.87      0.79      1749
           2       0.70      0.65      0.67       702

    accuracy                           0.70      3164
   macro avg       0.68      0.53      0.57      3164
weighted avg       0.69      0.70      0.68      3164



In [16]:
# model with test data
y_pred = rfc.predict( tfidfconverter.transform(test['message']))

In [17]:
#creat sentiment from the predicts
test['sentiment'] = y_pred
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,2
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


In [18]:
# convert output to csv
test[['tweetid','sentiment']].to_csv('tebogo_mokgonyana_Log_regresssion_submission_1.csv', index=False)